In the following notebook we present the code we used for the project 
### COMPONENT-WISE PERFORMANCE EVALUATION OF NEURAL NETWORKS USING THE SHAPLEY VALUE

Group members:
Marco Bagatella, Gianluca Lain, Samuele Giuliano Piazzetta, Andrea Ziani

Course: Introduction to Game Theory 2020

In [0]:
#standard imports
import tensorflow as tf
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization
from tensorflow.keras import backend as K
from itertools import chain, combinations


# list of players/blocks/components/strategies/...
players = ['bn', 'dropout', 'conv', 'activation', 'normalize', 'regularize', 'adam']
n_players = len(players)

# parameters that should be kept fixed
batch_size = 128
num_classes = 10
epochs = 20
input_shape = (28, 28, 1)

In [0]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [0]:
# methods building Keras models according to the active components
def get_lenet(active_players):
    kr = None and 'regularize' in active_players or tf.keras.regularizers.l1(0.01)
    inputs = tf.keras.layers.Input(shape=input_shape)

    if 'conv' in active_players:
        x = Conv2D(filters=6, kernel_size=5, kernel_regularizer=kr, padding='same')(inputs)
        if 'bn' in active_players:
            x = BatchNormalization()(x)
        if 'activation' in active_players:
            x = Activation(tf.keras.activations.tanh)(x)
        if 'dropout' in active_players:
            x = Dropout(0.2)(x)
        y = MaxPooling2D(2)(x)
        x = Conv2D(filters=16, kernel_size=5, kernel_regularizer=kr, padding='same')(y)
        if 'bn' in active_players:
            x = BatchNormalization()(x)
        if 'activation' in active_players:
            x = Activation(tf.keras.activations.tanh)(x)
        if 'dropout' in active_players:
            x = Dropout(0.2)(x)
        x = MaxPooling2D(2)(x)
        x = Conv2D(120, kernel_size=5, kernel_regularizer=kr, padding='same')(x)
        if 'bn' in active_players:
            x = BatchNormalization()(x)
        if 'activation' in active_players:
            x = Activation(tf.keras.activations.tanh)(x)
        if 'dropout' in active_players:
            x = Dropout(0.2)(x)
        x = Flatten()(x)
        x = Dense(86)(x)
        x = Dense(num_classes)(x)
        x = Activation(tf.keras.activations.softmax)(x)
    else:
        x = Flatten()(inputs)
        x = Dense(28*28, kernel_regularizer=kr)(x)
        if 'bn' in active_players:
            x = BatchNormalization()(x)
        if 'activation' in active_players:
            x = Activation(tf.keras.activations.tanh)(x)
        if 'dropout' in active_players:
            x = Dropout(0.2)(x)
        x = Dense(14*14, kernel_regularizer=kr)(x)
        if 'bn' in active_players:
            x = BatchNormalization()(x)
        if 'activation' in active_players:
            x = Activation(tf.keras.activations.tanh)(x)
        if 'dropout' in active_players:
            x = Dropout(0.2)(x)
        x = Dense(120, kernel_regularizer=kr)(x)
        if 'bn' in active_players:
            x = BatchNormalization()(x)
        if 'activation' in active_players:
            x = Activation(tf.keras.activations.tanh)(x)
        if 'dropout' in active_players:
            x = Dropout(0.2)(x)
        x = Dense(86)(x)
        x = Dense(num_classes)(x)
        x = Activation(tf.keras.activations.softmax)(x)

    return tf.keras.Model(inputs=inputs, outputs=x)


# adapted from official keras implementation
def get_vgg16(active_players, truncate=False):

    def vgg_block(x, img_size, filters, n_conv, active_players):
        kr = ('regularize' in active_players) and tf.keras.regularizers.l1(0.01) or None
        for i in range(n_conv):
            print('in')
            if 'conv' in active_players:
                x = Conv2D(filters, (3, 3), padding='same', kernel_regularizer=kr)(x)
            else:
                x = Dense(filters*img_size*img_size, kernel_regularizer=kr)(x)
            if 'bn' in active_players:
                x = BatchNormalization()(x)
            if 'activation' in active_players:
                x = Activation(tf.keras.activations.relu)(x)
            if 'dropout' in active_players:
                x = Dropout(0.2)(x)
        print('out')

        if 'conv' in active_players:
            x = layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
        return x

    inputs = tf.keras.layers.Input(shape=input_shape)
    if not 'conv' in active_players:
        x = Flatten()(inputs)
    else:
        x = inputs
    x = vgg_block(x, 28, 64, 2, active_players)
    x = vgg_block(x, 14, 128, 2, active_players)
    x = vgg_block(x, 7, 256, 3, active_players)
    if not truncate:
        x = vgg_block(x, 4, 512, 3, active_players)
        x = vgg_block(x, 2, 512, 3, active_players)

    if 'conv' in active_players:
        x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x)

    return tf.keras.Model(inputs, x, name='vgg16')

In [0]:
#takes a list of active components, builds the correct model, trains it and returns its (test) accuracy
def train_fixed_model(active_players, get_network, dataset=mnist, verbose=False):

    # check if one of the active players is not supported by the current model
    supported_players = ['bn', 'dropout', 'conv', 'activation', 'normalize', 'regularize', 'adam']
    if [player for player in active_players if player not in supported_players]:
        raise Exception("Some players are not supported")

    # load data and reshape it to 4d tensor
    # if MNIST is too easy we can switch to fashionMNIST
    (x_train, y_train), (x_test, y_test) = dataset.load_data()
    x_train = x_train.reshape(x_train.shape[0], input_shape[0], input_shape[1], input_shape[2]).astype('float32')
    x_test = x_test.reshape(x_test.shape[0], input_shape[0], input_shape[1], input_shape[2]).astype('float32')

    if 'normalize' in active_players:
        x_train /= 255
        x_test /= 255

    # convert class vectors to binary class matrices
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)

    # retrieve Keras model to fit
    model = get_network(active_players)

    opt = ('adam' in active_players) and'Adam' or 'SGD'
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=['accuracy'])
    #model.summary()

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=verbose,
              validation_data=(x_test, y_test), 
              callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)])

    score = model.evaluate(x_test, y_test)

    #only return the model's accuracy (score[0] contains the validation loss)
    return score[1]

In [0]:
# first of all we compute all possible coalitions/subset of players
performance = {}
for active_players in powerset(players):
    print('Computing utility of coalition ' + str(active_players))
    # for each coalition we compite the accuracy of the corresponding neural network
    performance[active_players] = train_fixed_model(active_players, get_lenet, dataset=mnist)

# we now have a dictionary associating a key (representing a coalition/tuple of active players) to the utility of the coalition v(S)
# from it we should be able to compute shapley values

Computing utility of coalition ()
313/313 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.0980
Computing utility of coalition ('bn',)
313/313 [==============================] - 1s 4ms/step - loss: 2.3235 - accuracy: 0.6658
Computing utility of coalition ('dropout',)
313/313 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.0980
Computing utility of coalition ('conv',)
313/313 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.0980
Computing utility of coalition ('activation',)
313/313 [==============================] - 1s 4ms/step - loss: 2.3919 - accuracy: 0.8112
Computing utility of coalition ('normalize',)
313/313 [==============================] - 1s 4ms/step - loss: 2.0839 - accuracy: 0.3751
Computing utility of coalition ('regularize',)
313/313 [==============================] - 1s 4ms/step - loss: nan - accuracy: 0.0980
Computing utility of coalition ('adam',)
313/313 [==============================] - 1s 4ms/

In [0]:
shapley_v = {}
for p in players:
    shapley_v[p] = 0

from math import factorial as fact
for p in players:
  for coalition in powerset(players):
        if coalition.count(p) != 0:
          S = len(coalition)
          coalition_p_missing = list(coalition)
          coalition_p_missing.remove(p)
          coalition_p_missing = tuple(coalition_p_missing)
          coeff = (fact(S - 1) * fact(n_players - S)) / fact(n_players)
          shapley_v[p] += coeff * (performance[coalition] - performance[coalition_p_missing])



tot = 0
for p in players:
    print('- ',str(p) + ' has a shapley value of ' + str(shapley_v[p]))
    tot += shapley_v[p]

print('- sum of shapley values: ' + str(tot))

print('\n\nResults vector to add in the tables below\n', [[s] for s in shapley_v.values()])

-  bn has a shapley value of 0.1936514155318339
-  dropout has a shapley value of -0.033933574954668685
-  conv has a shapley value of 0.1356114268302917
-  activation has a shapley value of 0.1732047678281862
-  normalize has a shapley value of 0.046469754104812945
-  regularize has a shapley value of -0.004185239970684043
-  adam has a shapley value of 0.2681814274440208
- sum of shapley values: 0.7789999768137927


Results vector to add in the tables below
 [[0.1936514155318339], [-0.033933574954668685], [0.1356114268302917], [0.1732047678281862], [0.046469754104812945], [-0.004185239970684043], [0.2681814274440208]]


In [0]:
import numpy as np
shapley_cardinality = {}
for c in range(8):
    shapley_cardinality[c] = []


# calculating mean and std of accuracy of each cardinality

for coalition in powerset(players):
    S = len(coalition)
    shapley_cardinality[S].append(performance[coalition])

mean_vec = []

for num, val in shapley_cardinality.items():
  print('- Cardinality ', num, ' mean: ', np.mean(val),'\t std: ', np.std(val))
  mean_vec.append([np.mean(val)])

print('Vector to append below: ', mean_vec)

- Cardinality  0  mean:  0.09799999743700027 	 std:  0.0
- Cardinality  1  mean:  0.4335714227386883 	 std:  0.3262560366800473
- Cardinality  2  mean:  0.6108857127172607 	 std:  0.33648423565326874
- Cardinality  3  mean:  0.7941600001284055 	 std:  0.20855257649924402
- Cardinality  4  mean:  0.8368342859404427 	 std:  0.19192936416753031
- Cardinality  5  mean:  0.8855285715489161 	 std:  0.12175209016114455
- Cardinality  6  mean:  0.8978428670338222 	 std:  0.043954008977941675
- Cardinality  7  mean:  0.8769999742507935 	 std:  0.0
Vector to append below:  [[0.09799999743700027], [0.4335714227386883], [0.6108857127172607], [0.7941600001284055], [0.8368342859404427], [0.8855285715489161], [0.8978428670338222], [0.8769999742507935]]


#Cardinality of previuos runs
Run 6 in MNIST:
- Cardinality  0  mean:  0.09799999743700027 	 std:  0.0
- Cardinality  1  mean:  0.4166428563850267 	 std:  0.3079052646046994
- Cardinality  2  mean:  0.6399476240788188 	 std:  0.30772990267273154
- Cardinality  3  mean:  0.7593428622399058 	 std:  0.257604142281983
- Cardinality  4  mean:  0.8630028537341526 	 std:  0.1621489308921093
- Cardinality  5  mean:  0.8604285745393663 	 std:  0.17886710161352726
- Cardinality  6  mean:  0.8436857206480843 	 std:  0.08622421424512326
- Cardinality  7  mean:  0.8482999801635742 	 std:  0.0

Run 7 in MNIST:
- Cardinality  0  mean:  0.09799999743700027 	 std:  0.0
- Cardinality  1  mean:  0.42590000693287167 	 std:  0.30256829158085485
- Cardinality  2  mean:  0.6243285724804515 	 std:  0.31322617610100734
- Cardinality  3  mean:  0.7759200004594666 	 std:  0.24348824669528593
- Cardinality  4  mean:  0.8515828566891807 	 std:  0.18358201521418865
- Cardinality  5  mean:  0.8786714332444328 	 std:  0.10909519691749131
- Cardinality  6  mean:  0.8226857015064785 	 std:  0.15482796621891487
- Cardinality  7  mean:  0.9258000254631042 	 std:  0.0

Run 8 in MNIST:
- Cardinality  0  mean:  0.09799999743700027 	 std:  0.0
- Cardinality  1  mean:  0.4335714227386883 	 std:  0.3262560366800473
- Cardinality  2  mean:  0.6108857127172607 	 std:  0.33648423565326874
- Cardinality  3  mean:  0.7941600001284055 	 std:  0.20855257649924402
- Cardinality  4  mean:  0.8368342859404427 	 std:  0.19192936416753031
- Cardinality  5  mean:  0.8855285715489161 	 std:  0.12175209016114455
- Cardinality  6  mean:  0.8978428670338222 	 std:  0.043954008977941675
- Cardinality  7  mean:  0.8769999742507935 	 std:  0.0

Run 9 in MNIST:

- Cardinality  0  mean:  0.09799999743700027 	 std:  0.0
- Cardinality  1  mean:  0.43552857105221066 	 std:  0.3252349475299099
- Cardinality  2  mean:  0.6720523859063784 	 std:  0.29223815093394856
- Cardinality  3  mean:  0.7679257122533661 	 std:  0.23592947538350084
- Cardinality  4  mean:  0.8725828494344439 	 std:  0.15697264670097694
- Cardinality  5  mean:  0.8810952419326419 	 std:  0.1283207374941204
- Cardinality  6  mean:  0.847000002861023 	   std:  0.08098461772288343
- Cardinality  7  mean:  0.9424999952316284 	 std:  0.0

Run 10 in MNIST:
- Cardinality  0  mean:  0.09799999743700027 	 std:  0.0
- Cardinality  1  mean:  0.4511571430734226 	 std:  0.3358043770828336
- Cardinality  2  mean:  0.6338000024358431 	 std:  0.3088663133367466
- Cardinality  3  mean:  0.7759457156062126 	 std:  0.24034221985490212
- Cardinality  4  mean:  0.8490314330373492 	 std:  0.19413545611245495
- Cardinality  5  mean:  0.884842859847205 	   std:  0.1279110222295204
- Cardinality  6  mean:  0.9156285609517779 	 std:  0.04990884608396854
- Cardinality  7  mean:  0.9451000094413757 	 std:  0.0

Run 6 in FashionMNIST:
- Cardinality  0  mean:  0.10000000149011612 	 std:  0.0
- Cardinality  1  mean:  0.31597142453704563 	 std:  0.2735630882921876
- Cardinality  2  mean:  0.536914291481177 	   std:  0.25042390324172126
- Cardinality  3  mean:  0.6919428580573627 	 std:  0.15823183541356742
- Cardinality  4  mean:  0.7205371465001787 	 std:  0.14044308544005568
- Cardinality  5  mean:  0.7791095205715725 	 std:  0.07333599831171173
- Cardinality  6  mean:  0.820028577532087 	   std:  0.042964898762224375
- Cardinality  7  mean:  0.8434000015258789 	 std:  0.0

Run 7 in FashionMNIST:
- Cardinality  0  mean:  0.10000000149011612 	 std:  0.0
- Cardinality  1  mean:  0.37204285923923763 	 std:  0.264404240070336
- Cardinality  2  mean:  0.5796000017296701 	 std:  0.2398798280604626
- Cardinality  3  mean:  0.6725714264171464 	 std:  0.17670380718774467
- Cardinality  4  mean:  0.7588628547532218 	 std:  0.08947848357310208
- Cardinality  5  mean:  0.7868857156662714 	 std:  0.07114331641307149
- Cardinality  6  mean:  0.8155857154301235 	 std:  0.04309452945289476
- Cardinality  7  mean:  0.8600999712944031 	 std:  0.0

Run 8 in FashionMNIST:
- Cardinality  0  mean:  0.10000000149011612 	 std:  0.0
- Cardinality  1  mean:  0.36928571867091314 	 std:  0.26963063298507023
- Cardinality  2  mean:  0.5144142842008954 	 std:  0.28183249235601743
- Cardinality  3  mean:  0.6650885690535818 	 std:  0.18649607726885817
- Cardinality  4  mean:  0.7477571427822113 	 std:  0.12112481555198364
- Cardinality  5  mean:  0.7890523814019703 	 std:  0.0752883124045957
- Cardinality  6  mean:  0.81554285969053 	 std:  0.05577976528109631
- Cardinality  7  mean:  0.8029999732971191 	 std:  0.0

Run 9 in FashionMNIST: 
- Cardinality  0  mean:  0.10000000149011612 	 std:  0.0
- Cardinality  1  mean:  0.3465285673737526 	 std:  0.2661457349498834
- Cardinality  2  mean:  0.5623666683123225 	 std:  0.24028737791753696
- Cardinality  3  mean:  0.6678228580525943 	 std:  0.19083674460856487
- Cardinality  4  mean:  0.7353342839649746 	 std:  0.1323979075692948
- Cardinality  5  mean:  0.7796904813675654 	 std:  0.0818871737175378
- Cardinality  6  mean:  0.8030285664967128 	 std:  0.0729062361429779
- Cardinality  7  mean:  0.8511000275611877 	 std:  0.0

Run 10 in FashionMNIST: 
- Cardinality  0  mean:  0.10000000149011612 	 std:  0.0
- Cardinality  1  mean:  0.3866000037108149 	 std:  0.2676601855123316
- Cardinality  2  mean:  0.5759238143052373 	 std:  0.24565775665638287
- Cardinality  3  mean:  0.6784542892660413 	 std:  0.1932352362479046
- Cardinality  4  mean:  0.7385257124900818 	 std:  0.1192412429516023
- Cardinality  5  mean:  0.7757380973725092 	 std:  0.07261977339948196
- Cardinality  6  mean:  0.823485723563603 	   std:  0.05226331339484322
- Cardinality  7  mean:  0.8184999823570251 	 std:  0.0

#Summary of previous runs

Run 1 on MNIST:

- bn has a shapley value of 0.1829752405307123
- dropout has a shapley value of -0.051969761294978004
- conv has a shapley value of 0.10841690983091082
- activation has a shapley value of 0.14445524064912685
- normalize has a shapley value of 0.009836909653884958
- regularize has a shapley value of -0.03509643092041923
- adam has a shapley value of 0.22668191242430896
- sum of shapley values: 0.5853000208735466

Run 2 on MNIST:

- bn has a shapley value of 0.13357499942538287
- dropout has a shapley value of -0.018326670002369646
- conv has a shapley value of 0.14341333104031426
- activation has a shapley value of 0.13583333398259828
- normalize has a shapley value of 0.095228326870572
- regularize has a shapley value of 0.0447616566504751
- adam has a shapley value of 0.27111500208931294
- sum of shapley values: 0.8055999800562859

Run 3 on MNIST:

- bn has a shapley value of 0.22754238260289034
- dropout has a shapley value of -0.036974279085795085
- conv has a shapley value of 0.1504190524419149
- activation has a shapley value of 0.1698157252122959
- normalize has a shapley value of -0.006735958531498902
- regularize has a shapley value of 0.01972738405068715
- adam has a shapley value of 0.29280571055909005
- sum of shapley values: 0.8166000172495844

Run 4 on MNIST:

- bn has a shapley value of 0.20629810667818502
- dropout has a shapley value of -0.03176024442627316
- conv has a shapley value of 0.1462147570082119
- activation has a shapley value of 0.13912808921720302
- normalize has a shapley value of 0.011001432704783619
- regularize has a shapley value of 0.02807976170664742
- adam has a shapley value of 0.2751380952873401
- sum of shapley values: 0.7740999981760979

Run 5 on MNIST:

- bn has a shapley value of 0.23558714317069177
- dropout has a shapley value of -0.030899522559983364
- conv has a shapley value of 0.126422146104631
- activation has a shapley value of 0.12720547792102607
- normalize has a shapley value of 0.029327134060717762
- regularize has a shapley value of 0.008858808520294371
- adam has a shapley value of 0.2556988127707015
- sum of shapley values: 0.7521999999880791

Run 6 on MNIST:
-  bn has a shapley value of 0.19044904508406207
-  dropout has a shapley value of -0.022257617754595616
-  conv has a shapley value of 0.12836403832549143
-  activation has a shapley value of 0.12866071013822453
-  normalize has a shapley value of 0.034092385552468706
-  regularize has a shapley value of 0.034507372833433615
-  adam has a shapley value of 0.25648404854748924
-  sum of shapley values: 0.7502999827265739

Run 7 on MNIST:
-  bn has a shapley value of 0.1517357234266542
-  dropout has a shapley value of 0.0422257172919455
-  conv has a shapley value of 0.12744071859688982
-  activation has a shapley value of 0.1453757195777835
-  normalize has a shapley value of 0.08255404721768128
-  regularize has a shapley value of -0.021365952207928602
-  adam has a shapley value of 0.299834054123078
-  sum of shapley values: 0.8278000280261038

Run 8 in MNIST:
-  bn has a shapley value of 0.1936514155318339
-  dropout has a shapley value of -0.033933574954668685
-  conv has a shapley value of 0.1356114268302917
-  activation has a shapley value of 0.1732047678281862
-  normalize has a shapley value of 0.046469754104812945
-  regularize has a shapley value of -0.004185239970684043
-  adam has a shapley value of 0.2681814274440208
-  sum of shapley values: 0.7789999768137927

Run 9 on MNIST:

-  bn has a shapley value of 0.21041428615294744
-  dropout has a shapley value of -0.0028623783872241057
-  conv has a shapley value of 0.12947595374924795
-  activation has a shapley value of 0.19077262351555493
-  normalize has a shapley value of 0.06287594804806372
-  regularize has a shapley value of 0.01476595259848095
-  adam has a shapley value of 0.2390576121175573
- sum of shapley values: 0.8444999977946281

Run 10 on MNIST:

-  bn has a shapley value of 0.24319452279735185
-  dropout has a shapley value of -0.04036880461942582
-  conv has a shapley value of 0.14785119018384385
-  activation has a shapley value of 0.16369453404276144
-  normalize has a shapley value of 0.05577952845820356
-  regularize has a shapley value of 0.0012828550878025264
-  adam has a shapley value of 0.2756661860538381
-  sum of shapley values: 0.8471000120043756

Run 1 on FashionMNIST:

- bn has a shapley value of 0.10732190280797933
- dropout has a shapley value of -0.014233092396032246
- conv has a shapley value of 0.1428269059175537
- activation has a shapley value of 0.07084191461049376
- normalize has a shapley value of 0.1311069048408951
- regularize has a shapley value of 0.02255024321022488
- adam has a shapley value of 0.248985241548646
- sum of shapley values: 0.7094000205397605

Run 2 on FashionMNIST:

- bn has a shapley value of 0.1766183383053258
- dropout has a shapley value of -0.034209997526236935
- conv has a shapley value of 0.12464999561863287
- activation has a shapley value of 0.07305833752311411
- normalize has a shapley value of 0.1675166693116937
- regularize has a shapley value of -0.009748334756919311
- adam has a shapley value of 0.239814999380282
- sum of shapley values: 0.7377000078558923

Run 3 on FashionMNIST:

- bn has a shapley value of 0.12678214057925202
- dropout has a shapley value of -0.011991188852559953
- conv has a shapley value of 0.14714048636101543
- activation has a shapley value of 0.09055714328728973
- normalize has a shapley value of 0.15673882111552215
- regularize has a shapley value of -0.004862853813739049
- adam has a shapley value of 0.25103547567767764
- sum of shapley values: 0.7554000243544579

Run 4 on FashionMNIST:

- bn has a shapley value of 0.12900785376273444
- dropout has a shapley value of -0.00829714181877318
- conv has a shapley value of 0.15401285918695584
- activation has a shapley value of 0.09680952300273239
- normalize has a shapley value of 0.1378178608382031
- regularize has a shapley value of -0.007742153285514738
- adam has a shapley value of 0.24749119052929539
- sum of shapley values: 0.7490999922156332

Run 5 on FashionMNIST:

- bn has a shapley value of 0.1390347662160084
- dropout has a shapley value of -0.034520237014761986
- conv has a shapley value of 0.1607064325451141
- activation has a shapley value of 0.019061433036057725
- normalize has a shapley value of 0.16875809251907323
- regularize has a shapley value of 0.005684760035503477
- adam has a shapley value of 0.26567475889765074
- sum of shapley values: 0.7244000062346457

Run 6 on FashionMNIST:

-  bn has a shapley value of 0.11886547511177403
-  dropout has a shapley value of -0.012171191544759846
-  conv has a shapley value of 0.15635215185937423
-  activation has a shapley value of 0.05550714090821288
-  normalize has a shapley value of 0.1576754749353443
-  regularize has a shapley value of 0.012255475847494033
-  adam has a shapley value of 0.25491547291832306
-  sum of shapley values: 0.7434000000357628

Run 7 on FashionMNIST:

-  bn has a shapley value of 0.1461711834051779
-  dropout has a shapley value of -0.019700483339173458
-  conv has a shapley value of 0.13215952157264665
-  activation has a shapley value of 0.09645952459956922
-  normalize has a shapley value of 0.1546778433202277
-  regularize has a shapley value of 0.012271183622734886
-  adam has a shapley value of 0.23806119662310393
-  sum of shapley values: 0.760099969804287

Run 8 on FashionMNIST:

-  bn has a shapley value of 0.09604784667137119
-  dropout has a shapley value of -0.026527149407636556
-  conv has a shapley value of 0.16073118678870657
-  activation has a shapley value of 0.09196451673550265
-  normalize has a shapley value of 0.16312619144363072
-  regularize has a shapley value of -0.02932381001966341
-  adam has a shapley value of 0.2469811895950919
-  sum of shapley values: 0.7029999718070031

Run 9 on FashionMNIST:
-  bn has a shapley value of 0.09343262838111038
-  dropout has a shapley value of -0.02575071290844963
-  conv has a shapley value of 0.16876429010714805
-  activation has a shapley value of 0.10990762463992551
-  normalize has a shapley value of 0.15431261983300962
-  regularize has a shapley value of 0.005359291427192233
-  adam has a shapley value of 0.24507428459113553
-  sum of shapley values: 0.7511000260710717

Run 10 on FashionMNIST:
-  bn has a shapley value of 0.14316785922717484
-  dropout has a shapley value of -0.02248714922794274
-  conv has a shapley value of 0.1381045251197758
-  activation has a shapley value of 0.0936278531061751
-  normalize has a shapley value of 0.13182118327489922
-  regularize has a shapley value of -0.0027338182819741097
-  adam has a shapley value of 0.23699952764880095
-  sum of shapley values: 0.718499980866909

In [0]:
#compute statistics on measurements

import numpy as np
import matplotlib.pyplot as plt

mnist_v = np.array([
           [[0.1829752405307123], [-0.051969761294978004], [0.10841690983091082], [0.14445524064912685], [0.009836909653884958], [-0.03509643092041923], [0.22668191242430896]],
           [[0.13357499942538287], [-0.018326670002369646], [0.14341333104031426], [0.13583333398259828], [0.095228326870572], [0.0447616566504751], [0.27111500208931294]],
           [[0.22754238260289034], [-0.036974279085795085], [0.1504190524419149], [0.1698157252122959], [-0.006735958531498902], [0.01972738405068715], [0.29280571055909005]],
           [[0.20629810667818502], [-0.03176024442627316], [0.1462147570082119], [0.13912808921720302], [0.011001432704783619], [0.02807976170664742], [0.2751380952873401]],
           [[0.23558714317069177], [-0.030899522559983364], [0.126422146104631], [0.12720547792102607], [0.029327134060717762], [0.008858808520294371], [0.2556988127707015]],
           [[0.19044904508406207], [-0.022257617754595616], [0.12836403832549143], [0.12866071013822453], [0.034092385552468706], [0.034507372833433615], [0.25648404854748924]],
           [[0.1517357234266542], [0.0422257172919455], [0.12744071859688982], [0.1453757195777835], [0.08255404721768128], [-0.021365952207928602], [0.299834054123078]],
           [[0.1936514155318339], [-0.033933574954668685], [0.1356114268302917], [0.1732047678281862], [0.046469754104812945], [-0.004185239970684043], [0.2681814274440208]],
           [[0.21041428615294744], [-0.0028623783872241057], [0.12947595374924795], [0.19077262351555493], [0.06287594804806372], [0.01476595259848095], [0.2390576121175573]], 
           [[0.24319452279735185], [-0.04036880461942582], [0.14785119018384385], [0.16369453404276144], [0.05577952845820356], [0.0012828550878025264], [0.2756661860538381]]
           ])

fmnist_v = np.array([[[0.10732190280797933], [-0.014233092396032246], [0.1428269059175537], [0.07084191461049376], [0.1311069048408951], [0.02255024321022488], [0.248985241548646]],
                    [[0.1766183383053258], [-0.034209997526236935], [0.12464999561863287], [0.07305833752311411], [0.1675166693116937], [-0.009748334756919311], [0.239814999380282]],
                    [[0.12678214057925202], [-0.011991188852559953], [0.14714048636101543], [0.09055714328728973], [0.15673882111552215], [-0.004862853813739049], [0.25103547567767764]],
                    [[0.12900785376273444], [-0.00829714181877318], [0.15401285918695584], [0.09680952300273239], [0.1378178608382031], [-0.007742153285514738], [0.24749119052929539]],
                    [[0.1390347662160084], [-0.034520237014761986], [0.1607064325451141], [0.019061433036057725], [0.16875809251907323], [0.005684760035503477], [0.26567475889765074]],
                    [[0.11886547511177403], [-0.012171191544759846], [0.15635215185937423], [0.05550714090821288], [0.1576754749353443], [0.012255475847494033], [0.25491547291832306]],
                    [[0.1461711834051779], [-0.019700483339173458], [0.13215952157264665], [0.09645952459956922], [0.1546778433202277], [0.012271183622734886], [0.23806119662310393]],
                    [[0.09604784667137119], [-0.026527149407636556], [0.16073118678870657], [0.09196451673550265], [0.16312619144363072], [-0.02932381001966341], [0.2469811895950919]],
                    [[0.09343262838111038], [-0.02575071290844963], [0.16876429010714805], [0.10990762463992551], [0.15431261983300962], [0.005359291427192233], [0.24507428459113553]],
                    [[0.14316785922717484], [-0.02248714922794274], [0.1381045251197758], [0.0936278531061751], [0.13182118327489922], [-0.0027338182819741097], [0.23699952764880095]]
                     ])
features_name = ['Batch Normalization','Dropout','Convolution','Activation',  'Normalization', 'Regularization', 'Adam']


dataset_name = mnist_v


x = np.mean(dataset_name, axis=0).squeeze()
mini = np.min(dataset_name, axis=0).reshape((7))
maxi = np.max(dataset_name, axis=0).reshape((7))

indexes = list(range(len(x)))
indexes.sort(key=x.__getitem__)
indexes = indexes[::-1]

x = x[indexes]
mini = x - mini[indexes].reshape(1, 7)
maxi = maxi[indexes].reshape(1, 7) - x
yerr = np.concatenate((mini, maxi), axis=0)
features_name = np.array(features_name)[indexes]
#mini = x - mini.reshape(1, 7)
#maxi = maxi.reshape(1, 7) - x
#print('mean: ', x,'\n min: ', mini, '\n max: ', maxi)
for feat, mean, mi, ma in zip(features_name, x, mini.squeeze(), maxi.squeeze()):
  print('('+str(feat)+','+str(mean)+') += (0,'+str(ma)+') -= (0, '+str(mi)+')\n')


(Adam,0.2660662861416737) += (0,0.033767767981404295) -= (0, 0.03938437371736475)

(Batch Normalization,0.19754228654007117) += (0,0.04565223625728068) -= (0, 0.0639672871146883)

(Activation,0.15181462220847605) += (0,0.03895800130707888) -= (0, 0.02460914428744998)

(Convolution,0.13436295241117477) += (0,0.016056100030740122) -= (0, 0.02594604258026395)

(Normalization,0.042042950813968964) += (0,0.05318537605660304) -= (0, 0.048778909345467865)

(Regularization,0.009133616834878925) += (0,0.035628039815596174) -= (0, 0.04423004775529816)

(Dropout,-0.0227127135793368) += (0,0.0649384308712823) -= (0, 0.029257047715641204)



## Code for printing the table in LaTeX


### MNIST Shapley Value

In [0]:
features_name = ['Batch Normalization','Dropout','Convolution','Activation',  'Normalization', 'Regularization', 'Adam']


for feature_name, mean_vec in sorted(zip(features_name, mnist_v.T.squeeze())):
  print('\\multicolumn{1}{|l|}{%s}'%feature_name, *['& %.3f'% value for value in mean_vec], '\\\\') 

\multicolumn{1}{|l|}{Activation} & 0.144 & 0.136 & 0.170 & 0.139 & 0.127 & 0.129 & 0.145 & 0.173 & 0.191 & 0.164 \\
\multicolumn{1}{|l|}{Adam} & 0.227 & 0.271 & 0.293 & 0.275 & 0.256 & 0.256 & 0.300 & 0.268 & 0.239 & 0.276 \\
\multicolumn{1}{|l|}{Batch Normalization} & 0.183 & 0.134 & 0.228 & 0.206 & 0.236 & 0.190 & 0.152 & 0.194 & 0.210 & 0.243 \\
\multicolumn{1}{|l|}{Convolution} & 0.108 & 0.143 & 0.150 & 0.146 & 0.126 & 0.128 & 0.127 & 0.136 & 0.129 & 0.148 \\
\multicolumn{1}{|l|}{Dropout} & -0.052 & -0.018 & -0.037 & -0.032 & -0.031 & -0.022 & 0.042 & -0.034 & -0.003 & -0.040 \\
\multicolumn{1}{|l|}{Normalization} & 0.010 & 0.095 & -0.007 & 0.011 & 0.029 & 0.034 & 0.083 & 0.046 & 0.063 & 0.056 \\
\multicolumn{1}{|l|}{Regularization} & -0.035 & 0.045 & 0.020 & 0.028 & 0.009 & 0.035 & -0.021 & -0.004 & 0.015 & 0.001 \\


### FMNIST Shapley Value


In [0]:
features_name = ['Batch Normalization','Dropout','Convolution','Activation',  'Normalization', 'Regularization', 'Adam']


for feature_name, mean_vec in sorted(zip(features_name, fmnist_v.T.squeeze())):
  print('\\multicolumn{1}{|l|}{%s}'%feature_name, *['& %.3f'% value for value in mean_vec], '\\\\') 

\multicolumn{1}{|l|}{Activation} & 0.071 & 0.073 & 0.091 & 0.097 & 0.019 & 0.056 & 0.096 & 0.092 & 0.110 & 0.094 \\
\multicolumn{1}{|l|}{Adam} & 0.249 & 0.240 & 0.251 & 0.247 & 0.266 & 0.255 & 0.238 & 0.247 & 0.245 & 0.237 \\
\multicolumn{1}{|l|}{Batch Normalization} & 0.107 & 0.177 & 0.127 & 0.129 & 0.139 & 0.119 & 0.146 & 0.096 & 0.093 & 0.143 \\
\multicolumn{1}{|l|}{Convolution} & 0.143 & 0.125 & 0.147 & 0.154 & 0.161 & 0.156 & 0.132 & 0.161 & 0.169 & 0.138 \\
\multicolumn{1}{|l|}{Dropout} & -0.014 & -0.034 & -0.012 & -0.008 & -0.035 & -0.012 & -0.020 & -0.027 & -0.026 & -0.022 \\
\multicolumn{1}{|l|}{Normalization} & 0.131 & 0.168 & 0.157 & 0.138 & 0.169 & 0.158 & 0.155 & 0.163 & 0.154 & 0.132 \\
\multicolumn{1}{|l|}{Regularization} & 0.023 & -0.010 & -0.005 & -0.008 & 0.006 & 0.012 & 0.012 & -0.029 & 0.005 & -0.003 \\


#Info about cardinality

In [0]:
import numpy as np

mnist = np.array([
                  [[0.09799999743700027], [0.4166428563850267], [0.6399476240788188], [0.7593428622399058], [0.8630028537341526], [0.8604285745393663], [0.8436857206480843], [0.8482999801635742]],
                  [[0.09799999743700027], [0.42590000693287167], [0.6243285724804515], [0.7759200004594666], [0.8515828566891807], [0.8786714332444328], [0.8226857015064785], [0.9258000254631042]],
                  [[0.09799999743700027], [0.4335714227386883], [0.6108857127172607], [0.7941600001284055], [0.8368342859404427], [0.8855285715489161], [0.8978428670338222], [0.8769999742507935]],
                  [[0.09799999743700027], [0.4511571430734226], [0.6338000024358431], [0.7759457156062126], [0.8490314330373492], [0.884842859847205], [0.9156285609517779], [0.9451000094413757]],
                  [[0.09799999743700027], [0.43552857105221066], [0.6720523859063784], [0.7679257122533661], [0.8725828494344439], [0.8810952419326419], [0.847000002861023], [0.9424999952316284]]
                
                ])
#mean of runs on fashion_MNIST
fashion_mnist = np.array([
  [[0.10000000149011612], [0.31597142453704563], [0.536914291481177], [0.6919428580573627], [0.7205371465001787], [0.7791095205715725], [0.820028577532087], [0.8434000015258789]],
  [[0.10000000149011612], [0.37204285923923763], [0.5796000017296701], [0.6725714264171464], [0.7588628547532218], [0.7868857156662714], [0.8155857154301235], [0.8600999712944031]],
  [[0.10000000149011612], [0.36928571867091314], [0.5144142842008954], [0.6650885690535818], [0.7477571427822113], [0.7890523814019703], [0.81554285969053], [0.8029999732971191]],
  [[0.10000000149011612], [0.3465285673737526], [0.5623666683123225], [0.6678228580525943], [0.7353342839649746], [0.7796904813675654], [0.8030285664967128], [0.8511000275611877]],
  [[0.10000000149011612], [0.3866000037108149], [0.5759238143052373], [0.6784542892660413], [0.7385257124900818], [0.7757380973725092], [0.823485723563603], [0.8184999823570251]]
])



dataset_name = fashion_mnist


x = np.mean(dataset_name, axis=0).squeeze()
std = np.std(dataset_name, axis=0).squeeze()

for num, (mean, std) in enumerate(zip(x, std)):
  print('('+str(num)+','+str(mean)+') += (0,'+str(std)+') -= (0, '+str(std)+')\n')


for i in range(8):
  print(i, 'layers,')

(0,0.10000000149011612) += (0,0.0) -= (0, 0.0)

(1,0.35808571470635275) += (0,0.024659633982198607) -= (0, 0.024659633982198607)

(2,0.5538438120058605) += (0,0.024758203563135007) -= (0, 0.024758203563135007)

(3,0.6751760001693453) += (0,0.009533975799985436) -= (0, 0.009533975799985436)

(4,0.7402034280981337) += (0,0.012794676508345974) -= (0, 0.012794676508345974)

(5,0.7820952392759778) += (0,0.005029137511899324) -= (0, 0.005029137511899324)

(6,0.8155342885426112) += (0,0.006925601121817251) -= (0, 0.006925601121817251)

(7,0.8352199912071228) += (0,0.021240479197198656) -= (0, 0.021240479197198656)

0 layers,
1 layers,
2 layers,
3 layers,
4 layers,
5 layers,
6 layers,
7 layers,


In [0]:
features_name = ['Batch Normalization','Dropout','Convolution','Activation',  'Normalization', 'Regularization', 'Adam']


for num, mean_mnist, mean_fmnist in (zip(range(8), mnist.T.squeeze(), fashion_mnist.T.squeeze() )):
  print('\\multicolumn{1}{|l|}{%s component(s)}'%num, *['& %.3f'% value for value in mean_mnist],*['& %.3f'% value for value in mean_fmnist] , '\\\\') 

\multicolumn{1}{|l|}{0 component(s)} & 0.098 & 0.098 & 0.098 & 0.098 & 0.098 & 0.100 & 0.100 & 0.100 & 0.100 & 0.100 \\
\multicolumn{1}{|l|}{1 component(s)} & 0.417 & 0.426 & 0.434 & 0.451 & 0.436 & 0.316 & 0.372 & 0.369 & 0.347 & 0.387 \\
\multicolumn{1}{|l|}{2 component(s)} & 0.640 & 0.624 & 0.611 & 0.634 & 0.672 & 0.537 & 0.580 & 0.514 & 0.562 & 0.576 \\
\multicolumn{1}{|l|}{3 component(s)} & 0.759 & 0.776 & 0.794 & 0.776 & 0.768 & 0.692 & 0.673 & 0.665 & 0.668 & 0.678 \\
\multicolumn{1}{|l|}{4 component(s)} & 0.863 & 0.852 & 0.837 & 0.849 & 0.873 & 0.721 & 0.759 & 0.748 & 0.735 & 0.739 \\
\multicolumn{1}{|l|}{5 component(s)} & 0.860 & 0.879 & 0.886 & 0.885 & 0.881 & 0.779 & 0.787 & 0.789 & 0.780 & 0.776 \\
\multicolumn{1}{|l|}{6 component(s)} & 0.844 & 0.823 & 0.898 & 0.916 & 0.847 & 0.820 & 0.816 & 0.816 & 0.803 & 0.823 \\
\multicolumn{1}{|l|}{7 component(s)} & 0.848 & 0.926 & 0.877 & 0.945 & 0.942 & 0.843 & 0.860 & 0.803 & 0.851 & 0.818 \\
